In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
#!google-drive-ocamlfuse -cc
#!fusermount -u drive
#!mkdir -p drive
#!google-drive-ocamlfuse drive
import os
os.chdir("/content/drive/GPU/CS6208/")
os.getcwd()

In [0]:
!pip install -q keras

In [0]:
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model, load_model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import gc

## Base Model and Basic params

In [0]:
epochs = 200
batchSize = 64
trainSteps = 5798//batchSize +1
valSteps   =  783//batchSize +1
testSteps  =  354//batchSize +1
imgSize = 128


def ourModel(inputShape):
  
  num_classes = 4

  base_model = VGG16(include_top=False,input_shape=inputShape)
  x = base_model.output

  x = GlobalAveragePooling2D(name='glob_pool')(x)
  x = Dense(256, activation='relu', name='dense_1')(x)
  predictions = Dense(num_classes, activation='softmax', name='dense_F')(x)

  model = Model(inputs=base_model.input, outputs=predictions)
  return model


In [0]:
# Note that xTrain and xVal are normalized by /=255
xTrain = np.load('./data/xTrain.npy', mmap_mode='r')
yTrain = np.load('./data/yTrain.npy', mmap_mode='r')
xVal   = np.load('./data/xVal.npy', mmap_mode='r')
yVal   = np.load('./data/yVal.npy', mmap_mode='r')

categories = ['cat','dog','horse','person']
datasets = ['train','val','test']

In [0]:
model = ourModel((imgSize,imgSize,3))
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

earlyStop = EarlyStopping(monitor='val_loss', patience=2,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.5,
                                        patience=0,
                                        verbose=1,
                                        epsilon=0.0001,
                                        cooldown=0,
                                        min_lr=0)


datagenTrain = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True)
datagenTrain.fit(xTrain)
trainGen = datagenTrain.flow(xTrain,yTrain, batch_size=batchSize)

datagenVal = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True)
datagenVal.fit(xVal)
valGen = datagenVal.flow(xVal,yVal, batch_size=batchSize)

# Checkpointer
dest_path = './baseModel/'
if not os.path.exists(dest_path):
  os.makedirs(dest_path)
    
filepath='{}modelBasev3.{{epoch:03d}}-{{val_loss:.2f}}.hdf5'.format(dest_path)
checkpointer = ModelCheckpoint(filepath, 
                               #period=1,
                               save_weights_only=False, 
                               monitor='val_loss', mode='auto',
                               save_best_only=True,
                               verbose=1)


model.fit_generator(trainGen, steps_per_epoch=trainSteps,
                    epochs=epochs,
                    validation_data=valGen, validation_steps=valSteps,
                    callbacks=[checkpointer, earlyStop, reduce_lr],
                    workers=1)

# Score trained model
scores = model.evaluate(xVal, yVal, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

## Prune and Tune
Tool: Keras Surgeon with corrected code

(Source:https://github.com/BenWhetton/keras-surgeon; visited: 03/08/2018)

In [0]:
#Get conv and dense layer names (deepest layer first)
#for layer in reversed(model.layers):
#    print(layer.name)

layerNames = ['dense_1',
              'block5_conv3','block5_conv2','block5_conv1',
              'block4_conv3','block4_conv2','block4_conv1',
              'block3_conv3','block3_conv2','block3_conv1',
              'block2_conv2','block2_conv1',
              'block1_conv2','block1_conv1']

In [0]:
from kerassurgeon import identify
from kerassurgeon.operations import delete_channels

#Initializations
#===============
# Initialize from last tuned layer
filepathLastTuned  = './baseModel/modelBasev3.004-0.73.hdf5'
model = load_model(filepathLastTuned, compile=False)
filepathTuned  = filepathLastTuned

opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

scores = model.evaluate(xVal, yVal, verbose=0)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])
baseAccuracy = scores[1]

accThresh  = 0.03 # Pruning threshold (accuracy)
chThresh   = 20    # Pruning threshold (channel)
predAccuracy = baseAccuracy
bestAccuracy = baseAccuracy
batch_size = 64
num_classes = 4
epochs = 200

earlyStop = EarlyStopping(monitor='val_loss', patience=2,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.5,
                                        patience=0,
                                        verbose=1,
                                        epsilon=0.0001,
                                        cooldown=0,
                                        min_lr=0)

datagenTrain = ImageDataGenerator()
datagenTrain.fit(xTrain)
trainGen = datagenTrain.flow(xTrain,yTrain, batch_size=batchSize)

datagenVal = ImageDataGenerator()
datagenVal.fit(xVal)
valGen = datagenVal.flow(xVal,yVal, batch_size=batchSize)

In [0]:
for layerName in layerNames:
  print('==========')
  print('Pruning {}\'s channels'.format(layerName))
  
  currPrune = 0 # To include in name when saving pruned model
  
  dest_path = './models/m2p01_{}/'.format(layerName)
  if not os.path.exists(dest_path):
    os.makedirs(dest_path)
  
  # Reload the optimizer so that each layer starts with this lr
  opt = SGD(lr=0.001, momentum=0.9)
  
  while True:

    # Prune channels
    layer = model.get_layer(name=layerName)
    apoz = identify.get_apoz(model, layer, valGen)
    high_apoz_channels = identify.high_apoz(apoz)
    if len(high_apoz_channels) < chThresh: 
      print('Few channels left to prune: {} < chThresh {}'.format(len(high_apoz_channels), chThresh))
      break
    model = delete_channels(model, layer, high_apoz_channels, 
                            copy=False)

    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    #Manual rubbish collection
    gc.collect()

    scores = model.evaluate(xVal, yVal,batch_size=batchSize,verbose=0)
    print('model after pruning: [loss,acc] = [{:.3f},{:.3f}]\n'.format(scores[0],scores[1]))

    # Save pruned model before tuning
    currPrune +=1
    filepathPruned="{}modelPruned.{}.{:03d}-{:03d}.hdf5".format(dest_path,layerName,currPrune,len(high_apoz_channels))
    model.save(filepathPruned,include_optimizer=False)

    # Save pruned channel indices
    filepathPruned_idx = "{}modelPruned.{}.{:03d}-{:03d}_ch.npy".format(dest_path,layerName,currPrune,len(high_apoz_channels))
    np.save(filepathPruned_idx, high_apoz_channels)

    # Save predictions (pre-tune)
    pred = model.predict(xVal, batch_size=batchSize)
    filepathPruned_pred = "{}modelPruned.{}.{:03d}-{:03d}_pred.npy".format(dest_path,layerName,currPrune,len(high_apoz_channels))
    np.save(filepathPruned_pred, pred)

    
    model.fit_generator(trainGen, steps_per_epoch=trainSteps,
                    epochs=epochs,
                    validation_data=valGen, validation_steps=valSteps,
                    callbacks=[earlyStop, reduce_lr],
                    verbose = 1,
                    workers=1)
    
    scores = model.evaluate(xVal, yVal,batch_size=batchSize,verbose=0)
    print('model after retraining: [loss,acc] = [{:.3f},{:.3f}]\n'.format(scores[0],scores[1]))
    

    # Save tuned model after earlyStop
    filepathTuned      = "{}modelTuned.{}.{:03d}-{:03d}.hdf5".format(dest_path,layerName,currPrune,len(high_apoz_channels))
    model.save(filepathTuned,include_optimizer=False)

    # Save predictions (tuned)
    pred = model.predict(xVal, batch_size=batchSize)
    filepathTuned_pred = "{}modelTuned.{}.{:03d}-{:03d}_pred.npy".format(dest_path,layerName,currPrune,len(high_apoz_channels))
    np.save(filepathTuned_pred, pred)

    # Exit while-loop if predAccuracy performs worse than some threshold
    _, predAccuracy = model.evaluate(xVal, yVal,batch_size=batchSize,verbose=0)
    if predAccuracy > bestAccuracy:
      bestAccuracy = predAccuracy
    elif (bestAccuracy-predAccuracy) > accThresh:
      print('Accuracy fell too much: {:.3f} > accThresh {:.3f}'.format((bestAccuracy-predAccuracy), accThresh))
      break


**(continue where last left-off) (if applicable)**

Stopped after block2_conv1 (block1_conv2, block1_conv1 not pruned) Likely due to insufficient memory (I believe Keras Surgeon implementation has to build a graph on depedent nodes)